# Spec2Vec API – Quickstart

*In order to use omigami, you need an access token provided by DataRevenue.*
## Introduction

This is a short guide on how you can add GNPS library matches to your mass spectra - using Omigami's Spec2Vec API.

The API uses a Spec2Vec model that was trained on the entire GNPS spectral library. It embeds each of your spectra into a vector space and calculates the cosine similarity to all GNPS library spectra in this vector space. It then returns you the top library matches for each of your spectra. To learn more about Spec2Vec read our gentle introduction: [Spec2Vec: The Next Step in Mass Spectral Similarity Metrics](https://www.datarevenue.com/en-blog/spec2vec-mass-spectral-similarity-metric)

This notebook shows you how 
1. Specify any MGF file
2. Run a library search through the Spec2Vec API
3. Save the results as CSV.

# 1. (Down)load a MS/MS dataset (MGF format)

We'll use a small MS/MS dataset in the MGF format from [here](https://gnps-external.ucsd.edu/gnpslibrary/GNPS-COLLECTIONS-MISC.mgf).
You can also select any other from the [GNPS spectral library](https://gnps-external.ucsd.edu/gnpslibrary), or - of course - use your own.

*Note that you will need the Precursor_MZ field `PEPMASS` and the abundance pairs in your MGF file.*

In [1]:
# Load your own MS/MS dataset (and skip the next cell)
path_to_mgf <- '/path/to/local_dataset.mgf'

In [2]:
# OR download a small MS/MS dataset from GNPS, in the same directory as this notebook
url <- 'https://gnps-external.ucsd.edu/gnpslibrary/GNPS-COLLECTIONS-MISC.mgf'

path_to_mgf <- 'GNPS-COLLECTIONS-MISC.mgf' # use your prefered saving path here

download.file(url, path_to_mgf, method = "curl")

# 2. Query for the best matches with Spec2Vec

`Romigami` is a R wrapper which instantiates a virtual environment on which to install and call the Python `Omigami` package

`Spec2Vec` is a python wrapper which :
- Builds a json payload from the MGF file
- Calls the Spec2Vec API
- Formats the prediction results into readable dataframes

____
`match_spectra_from_path()` takes the following arguments:
- `n_best` sets the number of matches you'd like per spectrum (it is set to 10 by default).
- `include_metadata` selects the metadata fields that will be displayed in the resulting top matches.
- `ion_mode` selects the ionization mode of the measured spectra (positive or negative)

In the results dataframes, the input spectra can be identified by their number in the dataframes index, which refers to their order in the MGF file.  
*i.e.* `matches of spectrum 1` gives the spectrum_id and Spec2Vec scores of the library spectra matches, for the first spectrum in the MGF file.

For each spectrum in the MGF file, the library spectra matches are sorted according to their Spec2Vec similarity score (best is first).   
The following information about the predicted spectra are returned :
- `score`, the Spec2Vec similarity score between the input spectrum and the library spectrum
- `matches of spectrum #`, the spectrum_ID of the matched library spectra for the spectrum number # in the MGF file
- `metadata`, metadata fields chosen with `include_metadata` in the input

In [3]:
# # install devtools and romigami if not already installed
if("devtools" %in% installed.packages() == FALSE) {install.packages("devtools")}
devtools::install_github("omigami/romigami")

# import the romigami package
library("romigami")

In [4]:
# environment setup
omigami <- omigami_init()

In [5]:
client <- omigami$Spec2Vec(token="MY_TOKEN")

In [6]:
# Run Spec2Vec library search with your user token
spectra_matches <- client$match_spectra_from_path(
    mgf_path = path_to_mgf,
    n_best = 10,
    include_metadata = list("Smiles", "Compound_name"),
    ion_mode = "positive"
)

## 2.1 View results
A list of dataframes is returned. To look at a specific dataframe you can call:
```r
spectra_matches[[i]]  # 'i' refers to the ith spectrum in the MGF file input
```

In [7]:
spectra_matches[[1]]

,score,compound_name,smiles
,<list>,<chr>,<chr>
CCMSLIB00004751476,0.3307738,Thelephoric acid,O=C(C1=C2C3=C(C=C(O)C(O)=C3)O1)C4=C(OC5=C4C=C(O)C(O)=C5)C2=O
CCMSLIB00006415274,0.2610556,"2,3-dihydroxypropyl palmitate",O=C(OCC(O)CO)CCCCCCCCCCCCCCC
CCMSLIB00006415154,0.259576,"2,3-dihydroxypropyl palmitate",O=C(OCC(O)CO)CCCCCCCCCCCCCCC
CCMSLIB00006415037,0.2516562,"2,3-dihydroxypropyl palmitate",O=C(OCC(O)CO)CCCCCCCCCCCCCCC
CCMSLIB00006415250,0.251224,"2,3-dihydroxypropyl palmitate",O=C(OCC(O)CO)CCCCCCCCCCCCCCC
CCMSLIB00006415298,0.2501113,"2,3-dihydroxypropyl palmitate",O=C(OCC(O)CO)CCCCCCCCCCCCCCC
CCMSLIB00006415131,0.2486516,"2,3-dihydroxypropyl palmitate",O=C(OCC(O)CO)CCCCCCCCCCCCCCC
CCMSLIB00006415084,0.244627,"2,3-dihydroxypropyl palmitate",O=C(OCC(O)CO)CCCCCCCCCCCCCCC
CCMSLIB00006415107,0.2409814,"2,3-dihydroxypropyl palmitate",O=C(OCC(O)CO)CCCCCCCCCCCCCCC


# 3. Save results

Execute the following cell to save the results in a CSV file. For readability, each dataframe is saved in its own CSV file under the `matches` directory.

In [8]:
if (!dir.exists("matches")){
    dir.create("matches")
}
for (i in seq_along(spectra_matches)){
    matches <- data.frame(lapply(spectra_matches[i], as.character), stringsAsFactors=FALSE)
    write.csv(matches, sprintf("matches/spectrum_%s.csv", i))
}


___